In [37]:
import pandas as pd
import nltk
import re
import numpy as np
nltk.download('punkt')
nltk.download('stopwords')
from nltk.corpus import stopwords
from sklearn.metrics import classification_report
from sklearn.svm import SVC, LinearSVC
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score,f1_score
from sklearn.ensemble import RandomForestClassifier
from gensim import utils
from gensim.models.doc2vec import LabeledSentence,TaggedDocument
from gensim.models import Doc2Vec
import gensim

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [38]:
df_train = pd.read_excel('P1_training.xlsx')
df_test = pd.read_excel('P1_testing.xlsx')
print("Training Data")
label=df_train['label']
label_test=df_test['label']
print(label.value_counts())
print("Testing Data")
print(label_test.value_counts())

Training Data
1    736
2    661
0    263
Name: label, dtype: int64
Testing Data
1    303
2    298
0     82
Name: label, dtype: int64


In [39]:
def preprocessor(text):
  review=re.sub('\[[^]]*\]', ' ', text)
  review=re.sub('[^a-zA-z]', ' ', text)
  review=review.lower().split()
  review=[i for i in review if not i in set(stopwords.words('english'))]
  review= ' '.join(review)
  return review
df_train['sentence']=df_train['sentence'].apply(preprocessor)
df_test['sentence']=df_test['sentence'].apply(preprocessor)

In [40]:
sentence=df_train['sentence']
sentence_test=df_test['sentence']


In [41]:
temp_df=df_train
lis=[i for i in range(0,1660)]
temp_df['id']=lis
temp_df

,sentence,label,id
0,living concentration camp like atmosphere led ...,1,0
1,even nod blues brothers believe filmmakers eve...,1,1
2,park lord screenwriter karey kirkpatrick reali...,1,2
3,ginger perfect spunky opinionated soft heart f...,2,3
4,jane horrocks delivers lovely voice characteri...,2,4
...,...,...,...
1655,lin shae plays mary neighbor magda also appear...,2,1655
1656,steve martin took extended vacation facets mov...,2,1656
1657,much book spares tinseltown mockery although r...,2,1657
1658,writer star bowfinger offers masses plenty goo...,1,1658


In [42]:
temp_df_test=df_test
lis_test=[i for i in range(0,683)]
temp_df_test['id']=lis_test
temp_df_test

,sentence,label,id
0,even whole thing proves creative delusion one ...,2,0
1,quite sure handle sam deed starts explaining b...,1,1
2,ruby close friend gretchen cuz ya love story w...,2,2
3,happy accidents romantic comedy filtered twelv...,2,3
4,film stars thandie newton robbed much deserved...,2,4
...,...,...,...
678,somehow considerable suspension disbelief larr...,1,678
679,occasionally violence slightly uncomfortable o...,0,679
680,perhaps sensational gods monsters brendan fras...,2,680
681,earned nomination touching performance althoug...,2,681


In [43]:
texts = temp_df.to_dict('records') 
documents = [TaggedDocument(text['sentence'].split(), [text['id']])  for text in texts]   

In [44]:
from nltk.tokenize import word_tokenize
res=[]
for i in df_train['sentence']:
  res.append(word_tokenize(i))

res_test=[]
for i in df_test['sentence']:
  res_test.append(word_tokenize(i))

In [45]:
model = gensim.models.Doc2Vec(vector_size=100,alpha=0.025,min_alpha=0.025, min_count=1, epochs=40)
model.build_vocab(documents)
model.train(documents, epochs=model.iter, total_examples=model.corpus_count)

In [46]:

array=np.zeros((len(res),100))
for i in range(len(df_train['sentence'])):
  array[i,:]=model.docvecs[i].reshape((1,100))
doc=pd.DataFrame(array)

In [47]:
array=np.zeros((len(res_test),100))
for i in range(len(df_test['sentence'])):
  array[i,:]=model.docvecs[i].reshape((1,100))
doc_test=pd.DataFrame(array)

In [48]:
import warnings
warnings. filterwarnings('ignore') 

In [49]:
from sklearn.svm import SVC, LinearSVC
clf=LinearSVC(max_iter=500)
clf.fit(doc,label)
predictions=clf.predict(doc_test)
print (classification_report(label_test,predictions))
print("Accuracy is",accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.09      0.01      0.02        82
           1       0.47      0.54      0.50       303
           2       0.47      0.51      0.49       298

    accuracy                           0.46       683
   macro avg       0.34      0.35      0.34       683
weighted avg       0.42      0.46      0.44       683

Accuracy is 0.46266471449487556


In [50]:
from sklearn.linear_model import LogisticRegression
clf=LogisticRegression(max_iter=100)
clf.fit(doc,label)
predictions=clf.predict(doc_test)
print (classification_report(label_test,predictions))
print("Accuracy is",accuracy_score(predictions,label_test))

              precision    recall  f1-score   support

           0       0.00      0.00      0.00        82
           1       0.47      0.55      0.51       303
           2       0.47      0.49      0.48       298

    accuracy                           0.46       683
   macro avg       0.31      0.35      0.33       683
weighted avg       0.41      0.46      0.43       683

Accuracy is 0.45973645680819913
